`GB` BigData / [Олег Гладкий](https://gb.ru/users/3837199) // домашнее задание

`262698` __Методы сбора и обработки данных из сети Интернет__:  `02`. Парсинг даных: HTML, DOM, XPath


## Задание 1. XPath

In [1]:
import requests
from lxml import html

In [2]:
%%writefile dom_example.html
<!DOCTYPE HTML>
<html>
    <head>
        <title>Page_Tytle</title>
    </head>
    <body>
        <h1>
            Header-1
        </h1>
        <div>
            <h2>User Header, 1 any-2-text</h2>
            <p class='price price_color'>any-text any-text any-text</p>
            <p class='price price_2'>any-2-text any-2-text any-2-text</p>
        </div>
        <div class="super">
            <h2>User Header, 2</h2>
            <p class='price price_color'>text-3 text-3 text-3 text-3 text-3</p>
            <p class='price price_2'>any-4-text any-4-text any-4-text</p>
        </div>
        <div class="partner">
            <svg class="card-big__partners-badge">
                <use xlink:href="#ui-label_moslenta">
                </use>
            </svg>
        </div>
        
    </body>
</html>


Overwriting dom_example.html


In [61]:
# доступ к странице на диске через Jupyter не работает, 
# потому-что мы получаем страницу по идентификации Jupyter,
# а не нашу страничку...
# Нужно установить, видимо, локальный сервер, чтобы...

# url = 'http://localhost:8889/view/02-01-Data_Get_internet/02_XPath/home_work/dom_example.html'
# url = 'file:///X:/GeekBrains/02-01-Data_Get_internet/02_XPath/home_work/dom_example.html'
# responce = requests.get(url=url)
# responce.headers['content-type']

Попробуем подставить html-файл как `request.get(...).text` атрибут в метод `fromstring` для получения `DOM`

In [3]:
with open('dom_example.html', 'r') as f:
    responce = f.read()
    
dom = html.fromstring(responce)

In [4]:
dom.xpath('//p[@class="price price_color"]/text() | //p[@class="price price_2"]/text()')

['any-text any-text any-text',
 'any-2-text any-2-text any-2-text',
 'text-3 text-3 text-3 text-3 text-3',
 'any-4-text any-4-text any-4-text']

In [25]:
dom.xpath('//p[@class="price price_color"]/text()')

['any-text any-text any-text', 'text-3 text-3 text-3 text-3 text-3']

In [40]:
from scrapy.selector import Selector
body = '<html><body><span>good</span></body></html>'
Selector(text=body).xpath('//span/text()').getall()

['good']

In [43]:
body = responce
Selector(text=body).xpath('//p[@class="price price_color"]/text()').getall()

['any-text any-text any-text', 'text-3 text-3 text-3 text-3 text-3']

In [64]:
dom.xpath("//p[contains(@class, 'price')]")

[<Element p at 0x5826180>,
 <Element p at 0x5826db0>,
 <Element p at 0x58265e0>,
 <Element p at 0x5826e00>]

In [68]:
dom.xpath('.//div/*[name()="svg"]/*[name()="use"]/attribute::*')

['#ui-label_moslenta']

In [84]:
dom.xpath('.//svg/use/attribute::*')

['#ui-label_moslenta']

Спустимся, для установки точки `"any-2"`, а потом поднимимся к родителю и выберем из списка любой `1`-ый элемент

In [46]:
elements = dom.xpath('.//p[contains(text(), "any-2")]/../*[1]/text()')

print(f"Всего найдено: {len(elements)}\nСписок: \n")

for el in elements:
    print(el)

Всего найдено: 1
Список: 

User Header, 1 any-2-text


In [37]:
import datetime

In [45]:
str(dt.date.today())

'2022-09-30'

In [44]:
news_time = str(datetime.today().strftime("%H:%M"))

AttributeError: module 'datetime' has no attribute 'today'

## HH.ru /xpath
ВНИМАНИЕ
* Если после заголовка `h1` следует тег `span`, то его указывать не надо! Так как он куда-то пропадает...

In [6]:
import requests
from lxml import html

In [2]:
url = 'https://hh.ru/vacancy/73537275'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
}

response = requests.get(url=url, headers=headers)


response.status_code, response.headers['Content-Type'], response.url, 

(200, 'text/html; charset=utf-8', 'https://nahabino.hh.ru/vacancy/73537275')

In [7]:
dom = html.fromstring(response.text) 

In [25]:
title = dom.xpath('//div[@class="vacancy-title"]/h1/text()')
title

['Senior Python Developer, Удаленно']

In [43]:
dom.xpath('.//div[contains(@class, "block-employer")]//a[@class="vacancy-company-name"]/text()')


[]

In [61]:
dom.xpath('//a[@class="vacancy-company-name"]//span[@data-qa="bloko-header-2"]/span/text()')

[]

In [62]:
dom.xpath('//span[@data-qa="bloko-header-2"]/text()')

['Vocato', 'Vocato']

## HH.ru /BeautifulSoup

In [69]:
from bs4 import BeautifulSoup as bs

In [70]:
soup = bs(response.text, 'html.parser')

In [17]:
cards = soup.find(name='div', attrs={'class': ['vacancy-title']})

In [71]:
cards.find(name='h1')

<h1 class="bloko-header-section-1" data-qa="vacancy-title">Senior Python Developer, Удаленно</h1>

In [72]:
cards.find(name='h1').text

'Senior Python Developer, Удаленно'

In [81]:
soup.find(name='a', attrs={'data-qa': ['vacancy-company-name']}).find(name='span', attrs={'data-qa': ['bloko-header-2']}).text

'Vocato'

In [79]:
soup.find(name='span', attrs={'data-qa': ['bloko-header-2']}).text

'Vocato'